In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
from sklearn.metrics import accuracy_score

In [2]:
import os
import cv2
import numpy as np

In [3]:
data_dir = "C:/Users/priya/Downloads/archive (1)/Disaster_Dataset"

In [4]:
image_exts = ['jpeg','jpg', 'bmp', 'png', 'svg']

In [5]:
def load_dataset(dataset_dir):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_dict = {class_name: i for i, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                image = cv2.imread(image_path)  # Read image using OpenCV
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                image = cv2.resize(image, (224, 224))  # Resize image if needed
                images.append(image)
                labels.append(class_dict[class_name])

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, class_names

In [6]:
images, labels, class_names = load_dataset(data_dir)

In [7]:
# Print information about the dataset
print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Number of classes:", len(class_names))
print("Class names:", class_names)

Number of images: 4876
Number of labels: 4876
Number of classes: 12
Class names: ['drought', 'earthquake', 'human_damage', 'infrastructure', 'land_slide', 'non_damage_buildings_street', 'non_damage_human', 'non_damage_sea', 'non_damage_wildlife_forest', 'urban_fire', 'water_disaster', 'wild_fire']


In [8]:
# Step 2: Create the MobileNet Model
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
num_classes = 12
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Step 3: Compile the Model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
root_dir = data_dir
train_dataset = ImageFolder(root=root_dir, transform=None)

In [13]:
# Step 2: Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

In [15]:
train_dataset.transform = transform

In [16]:
# Step 3: Load the MobileNet model
mobilenet = mobilenet_v2(pretrained=True)

c:\Users\priya\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\priya\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\priya/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [17]:
# Step 4: Modify the classifier part of MobileNet
# In MobileNetV2, the classifier is the last layer (fc layer)
# We need to modify it to suit the number of classes in our dataset
num_classes = len(train_dataset.classes)
mobilenet.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(in_features=1280, out_features=num_classes)
)

In [18]:


# Step 5: Train the model
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    mobilenet.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = mobilenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:

# Step 6: Evaluate the model
mobilenet.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = mobilenet(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")